In [1]:
%load_ext autoreload
%autoreload 2
    
from scipy.stats import norm
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.base.model import GenericLikelihoodModel
import scipy.stats as stats
import sys

sys.path.append("../")
import vuong_tests10 as vuong_tests_fast
from vuong_test_base import *


In [2]:
class JointNormal1(GenericLikelihoodModel):
    
    def loglikeobs(self, params):
        data = np.concatenate([[self.endog],self.exog.transpose()],axis=0)
        mult_rv = stats.multivariate_normal([params[0], 0.0], [[1,0],[0,1]])
        return mult_rv.logpdf(data.transpose())
    
    
class JointNormal2(GenericLikelihoodModel):
    
    def loglikeobs(self, params):
        data = np.concatenate([[self.endog],self.exog.transpose()],axis=0)
        mult_rv = stats.multivariate_normal([0.0, params[0]], [[1,0],[0,1]])
        return mult_rv.logpdf(data.transpose())


def setup_shi(yn,xn):
    # model 1 grad, etc.
    nobs = yn.shape[0]
    model1_param = np.array([yn.mean()])
    model2_param = np.array([xn.mean()])
    
    model1_deriv = JointNormal1(yn,xn)
    ll1 = model1_deriv.loglikeobs(model1_param)
    grad1 =  model1_deriv.score_obs(model1_param).reshape( (nobs,1) )
    hess1 = model1_deriv.hessian(model1_param)
    
    
    model2_deriv = JointNormal2(yn,xn)
    ll2 = model2_deriv.loglikeobs(model2_param)
    grad2 =  model2_deriv.score_obs(model2_param).reshape( (nobs,1) )  
    hess2 = model2_deriv.hessian(model2_param)
    
    return ll1,grad1,hess1,model1_param,ll2,grad2,hess2,model2_param

def gen_data(beta= 1.5, nobs=1000):
    cov = [[25, 0], [0, 1]]
    data = np.random.multivariate_normal([beta,beta], [[25,0],[0,1]],  nobs)
    return data[:,0],data[:,1],nobs

yn,xn,nobs = gen_data()
ll1,grad1,hess1,params1,ll2,grad2,hess2,params2 = setup_shi(yn,xn)
print(grad1.shape,hess1.shape)
#NOTE! Weird size distortions with shi's test when theta = .5....

(1000, 1) (1, 1)


In [3]:
def sw_test_stat(ll1, grad1, hess1, params1, ll2, grad2, hess2, params2, epsilon=.5,compute_v=True,print_stuff=False):
    nobs = ll1.shape[0]

    idx_even = np.arange(0, nobs, 2)         # 0-based indices: 0,2,4,...
    idx_odd  = np.arange(1, nobs, 2)         # 1,3,5,...

    # Regular loglikelihood ratio statistic
    llr = (ll1 - ll2).sum()
    
    # Split-sample difference statistic
    llr_split = ll1[idx_even].sum() - ll2[idx_odd].sum()

    # Regularized numerator (as in your expression)
    llr_reg = llr + epsilon * llr_split

    # Main variance
    omega2 = (ll1 - ll2).var()
    
    # Split-group variances
    omega_A2 = ll1[idx_even].var()
    omega_B2 = ll2[idx_odd].var()

    # Regularized variance
    omega_reg2 = (1 + epsilon) * omega2 + (epsilon**2 / 2) * (omega_A2 + omega_B2)
    omega_reg = np.sqrt(omega_reg2)

    if print_stuff:
        print('llr',llr,llr_split)
        print('omega',omega2,omega_A2,omega_B2,np.sqrt(omega2),np.sqrt(omega_reg2))
    if not compute_v:
        return llr_reg,omega_reg,nobs 
        
    V = compute_eigen2(ll1, grad1, hess1, params1, ll2, grad2, hess2, params2) # If you want to try to bias-correct you can, this is optional
    return llr_reg,omega_reg,V,nobs 
        
    


def sw_test(ll1, grad1, hess1, params1, ll2, grad2, hess2, params2, epsilon=.5,
            alpha=.05,  biascorrect=False):

    llr_reg,omega_reg,V,nobs =  sw_test_stat(ll1, grad1, hess1, params1, ll2, grad2, hess2, params2, epsilon=.5)

    if biascorrect:
        llr_reg += V.sum() / 2

    test_stat = llr_reg / (omega_reg * np.sqrt(nobs))

    # Two-sided test
    reject_high = (test_stat >= norm.ppf(1 - alpha / 2))
    reject_low  = (test_stat <= norm.ppf(alpha / 2))
    return int(reject_high) + 2 * int(reject_low)

    
yn,xn,nobs = gen_data()
ll1,grad1,hess1,params1,ll2,grad2,hess2,params2 = setup_shi(yn,xn)
print(grad1.shape,hess1.shape)

(1000, 1) (1, 1)


In [4]:
def bootstrap_distr(ll1, grad1, hess1, params1, ll2, grad2, hess2, params2,
                    epsilon=0.5, trials=500, seed=None, biascorrect=False):
    nobs = ll1.shape[0]
    print('---------')
    llr_reg, omega_reg, V, nobs = sw_test_stat(
        ll1, grad1, hess1, params1, ll2, grad2, hess2, params2, epsilon=epsilon,print_stuff=True
    )
    if biascorrect:
        llr_reg += V.sum() / 2
    test_stat = llr_reg / (omega_reg * np.sqrt(nobs))
    stat_dist = []

    test_stat0 = (ll1-ll2).sum()/np.sqrt( nobs*(ll1-ll2).var() )
    stats_s0 = []
    
    
    for i in range(trials):
        if seed is not None:
            np.random.seed(seed + i)
        sample = np.random.choice(np.arange(nobs), nobs, replace=True)
        ll1_s = ll1[sample]
        ll2_s = ll2[sample]

        # Don't need V for bootstrap stats (saves computation)
        llr_reg_s, omega_reg_s, nobs_in_s = sw_test_stat(
            ll1_s, grad1, hess1, params1, ll2_s, grad2, hess2, params2, epsilon=epsilon, compute_v=False,print_stuff=(i==0)
        )


        
        # bias correct V not directly available for bootstrap
        stat_s = llr_reg_s / (omega_reg_s * np.sqrt(nobs_in_s))
        stat_dist.append(stat_s)

        ###### real stat for comparison...
        llrs0 = ll1_s - ll2_s
        stats_s0.append(  llrs0.sum()/  np.sqrt(nobs*llrs0.var()) )

    print('test_stat0', nobs_in_s, nobs, (np.array(stats_s0)-test_stat0).mean(),np.array(stats_s0).mean(),test_stat0 )
    print('test_stat', nobs_in_s, nobs, (np.array(stat_dist)-test_stat).mean(),np.array(stat_dist).mean(),test_stat )
    #print('----')
    return np.array(stat_dist)-test_stat, test_stat

def sw_bs_test_helper(stat_dist, stat_obs, alpha=.05, left=True, right=True, print_stuff=False):
    cv_upper = np.percentile(stat_dist, 100 * (1 - alpha / 2))
    cv_lower = np.percentile(stat_dist, 100 * (alpha / 2))
    if print_stuff:
        print(f"mean={stat_dist.mean():.3f}, cv_lower={cv_lower:.3f}, stat_obs={stat_obs:.3f}, cv_upper={cv_upper:.3f}")
    out = 0
    if right and (stat_obs > cv_upper):
        out = 1
    if left and (stat_obs < cv_lower):
        out += 2
    return out

def sw_bs_test(ll1, grad1, hess1, params1, ll2, grad2, hess2, params2,
               alpha=.05, trials=500, epsilon=0.5, biascorrect=False, seed=None, print_stuff=False):
    stat_dist, test_stat = bootstrap_distr(
        ll1, grad1, hess1, params1, ll2, grad2, hess2, params2,
        epsilon=epsilon, trials=trials, seed=seed, biascorrect=biascorrect
    )
    return sw_bs_test_helper(stat_dist, test_stat, alpha=alpha, print_stuff=print_stuff)

# Example Usage:
# result = sw_bs_test(ll1, grad1, hess1, params1, ll2, grad2, hess2, params2, epsilon=0.5, trials=1000)

In [5]:
def monte_carlo2(total, gen_data, setup_shi, alpha=.05,  c1=.5,trials=500, biascorrect=False):
    # Count for [model1, tie, model2] for each test
    reg = np.zeros(3, dtype=int)
    sw  = np.zeros(3, dtype=int)
    swbs = np.zeros(3, dtype=int)

    # Tracking statistics
    llr_sum = 0.0
    llr_sq_sum = 0.0
    omega_sum = 0.0
    nobs_last = None

    for _ in range(total):
        np.random.seed()
        yn, xn, nobs = gen_data()

        ll1, grad1, hess1, params1, ll2, grad2, hess2, params2 = setup_shi(yn, xn)

        # Update LLR-related stats
        llrn = (ll1 - ll2).sum()
        omegan = np.sqrt((ll1 - ll2).var())
        llr_sum   += llrn
        llr_sq_sum += llrn ** 2
        omega_sum += omegan
        nobs_last = nobs

        # Run the tests
        reg_idx = regular_test(
            ll1, grad1, hess1, params1,
            ll2, grad2, hess2, params2,
            biascorrect=biascorrect, alpha=alpha
        )
        sw_idx = sw_test(
            ll1, grad1, hess1, params1,
            ll2, grad2, hess2, params2, epsilon = c1,
            alpha=alpha, biascorrect=biascorrect
        )
        swbs_idx = sw_bs_test(
            ll1, grad1, hess1, params1,
            ll2, grad2, hess2, params2, epsilon = c1,
            alpha=alpha, biascorrect=biascorrect, print_stuff=False
        )

        reg[reg_idx]   += 1
        sw[sw_idx]     += 1
        swbs[swbs_idx] += 1

    # Convert counts to rates/frequencies
    reg_freq = reg / total
    sw_freq = sw / total
    swbs_freq = swbs / total

    llr_mean = llr_sum / total
    llr_sd   = np.sqrt(llr_sq_sum / total - llr_mean ** 2)
    omega_scaled = omega_sum * np.sqrt(nobs_last) / total if nobs_last is not None else np.nan

    return reg_freq, sw_freq, swbs_freq, llr_mean, llr_sd, omega_scaled


In [6]:
nobs = 250
num_sims = 100
c1 = .1
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn)

gen_data_ex = lambda : gen_data(nobs=nobs,beta=0)
mc_out = monte_carlo2(num_sims,gen_data_ex,setup_shi_ex,trials=500,c1=c1)
print(mc_out)

---------
llr 1.2706746079498872 65.09783811628267
omega 0.267323312793606 220.9109419122211 242.6552303915194 0.5170331834549945 1.6161331954983382
llr 0.5101353477309518 -99.21767906269156
omega 0.27143053824350744 256.4274089317372 221.45251615519973 0.5209899598298487 1.6395039547078083
test_stat0 250 250 -0.01563265233680399 0.139801314172346 0.15543396650915003
test_stat 250 250 -0.2209568843412182 0.08352259686285776 0.304479481204076
---------
llr 70.69892365759719 162.8363615187136
omega 14.64955072767355 402.7899308686772 321.95043611832915 3.8274731517900356 4.442770265878705
llr 54.95639332340723 405.60266704779815
omega 13.49312080360358 162.86339169537956 311.8849402555262 3.673298354830925 4.149237826844645
test_stat0 250 250 0.036664589797178064 1.20490074986435 1.1682361600671718
test_stat 250 250 -0.17306583619325816 1.065184126098409 1.2382499622916672
---------
llr 10.96972945164439 275.6022276430749
omega 2.4145272067937578 258.4392668966297 293.5069568355615 1.553

In [7]:
def gen_data3(beta= 1.5, nobs=1000):
    cov = [[25, 0], [0, 1]]
    data = np.random.multivariate_normal([0,beta], [[25,0],[0,1]],  nobs)
    return data[:,0],data[:,1],nobs

gen_data_ex = lambda : gen_data3(nobs=nobs,beta=.5)
mc_out = monte_carlo2(num_sims,gen_data_ex,setup_shi_ex,trials=500,c1=c1)
print(mc_out)

---------
llr -19.493492330866353 56.274713018482316
omega 2.103268697639458 327.28211086448863 232.02494881840823 1.4502650439279912 2.260559856720872
llr -12.225948029652457 353.75091506427475
omega 2.1056771425539402 154.38129133162568 383.6005264528247 1.4510951528255962 2.237443618447532
test_stat0 250 250 -0.02124954497913194 -0.8713539527858893 -0.8501044078067574
test_stat 250 250 -0.20612432449085138 -0.5940654657539225 -0.3879411412630711
---------
llr -26.6403194339536 210.89308742134563
omega 1.3613717796478135 256.5713701132727 318.6749463665136 1.1667783764056538 2.0913489761423194
llr -22.624929700025454 -556.3669079048002
omega 1.5898086726191083 440.16818253568044 242.48581706227097 1.26087615276803 2.272016623590324
test_stat0 250 250 0.0010848100472642486 -1.4429612694706142 -1.4440460795178784
test_stat 250 250 -0.6323324282633002 -0.800203379523427 -0.16787095126012672
---------
llr -21.98713064920992 -278.00532377430636
omega 0.22084949086411787 633.1576143113849 

# size

In [8]:
gen_data_ex = lambda : gen_data(nobs=nobs,beta=.5)
mc_out = monte_carlo2(num_sims,gen_data_ex,setup_shi_ex,trials=500,c1=c1)
print(mc_out)

---------
llr -9.34896894369145 -325.25281021596425
omega 2.9014965449152106 414.1332893513485 279.0476474856982 1.7033779806358924 2.580223029815827
llr -55.18957276751931 -353.23740817412977
omega 2.5863858289775363 489.368619717943 189.84384435081788 1.6082244336464784 2.4982167104194732
test_stat0 250 250 -0.023595024212583564 -0.3707174594246063 -0.3471224352120228
test_stat 250 250 0.7673410355530774 -0.25906640032651185 -1.0264074358795892
---------
llr 2.929265346779463 -209.9498347933029
omega 8.655290702395863 337.57992183854765 245.02952659829162 2.9419875428689126 3.5261688863155216
llr 33.895670445427406 163.15753445704422
omega 9.975682328239243 354.5804139426367 435.02863105606684 3.1584303583012945 3.8628093126708554
test_stat0 250 250 -0.0009055601087316188 0.062066496923114334 0.06297205703184595
test_stat 250 250 0.38380299054209455 0.059775306631821394 -0.3240276839102732
---------
llr -29.444877492321886 51.560733399415085
omega 2.103443013022205 371.89783884322395

In [9]:
gen_data_ex = lambda : gen_data(nobs=nobs,beta=1.5)
mc_out = monte_carlo2(num_sims,gen_data_ex,setup_shi_ex,trials=500,c1=c1)
print(mc_out)

---------
llr -23.112607449441896 51.013628770263495
omega 48.01749546014475 409.7959223611158 315.4106374113063 6.929465741321242 7.513007241113331
llr -154.0273214895097 -450.7086633388624
omega 45.23380007825055 364.9117978336078 203.51173929700076 6.725607785044453 7.25253733335642
test_stat0 250 250 0.016959288586108335 -0.19399054487587394 -0.21094983346198226
test_stat 250 250 -0.05200137002211495 -0.20362256381385269 -0.15162119379173772
---------
llr 208.5081302243714 134.96090238004012
omega 96.38090962428862 288.79900489932595 411.36222289863645 9.817377940381466 10.465171127397168
llr 40.04664909270119 379.1364869067188
omega 80.8188009299779 103.51480713719215 327.8690637153296 8.989927748874175 9.542410616675344
test_stat0 250 250 0.07076087341271593 1.414012821491204 1.3432519480784881
test_stat 250 250 0.045063285612690095 1.38673076201615 1.34166747640346
---------
llr 109.64415363578917 185.5918985847161
omega 72.56409023169236 229.77222368527623 607.1527581549299 8.5

In [10]:
gen_data_ex = lambda : gen_data(nobs=500,beta=.5)
mc_out = monte_carlo2(num_sims,gen_data_ex,setup_shi_ex,trials=500,c1=c1)
print(mc_out)

---------
llr 52.632141921595974 -361.0889368415096
omega 11.83347642908159 275.87031778110435 249.14088886761496 3.4399820390638074 3.95498168203512
llr -10.220310961855294 609.5622229105738
omega 11.436249543871385 171.4459867295442 321.313349830549 3.3817524368101495 3.878617173820973
test_stat0 500 500 -0.10816255662169018 0.5760796620971851 0.6842422187188751
test_stat 500 500 0.3452160992757466 0.5320544173183157 0.1868383180425691
---------
llr -59.46378420205431 -232.40033004693169
omega 0.2880141022188847 357.22411389099227 339.14934801389836 0.5366694534058042 1.9490209906425398
llr -38.61914136570205 25.80303682917838
omega 0.29482931522747097 345.18376771307595 361.69585816156797 0.5429818737559026 1.964360042386181
test_stat0 500 500 -0.1295938463372675 -5.084787879884379 -4.955194033547111
test_stat 500 500 0.45964849897349863 -1.438036114592795 -1.8976846135662937
---------
llr 52.61031454490916 -518.71968968501
omega 15.189099937345032 420.7132976713937 410.653945466716

# power 0.... need to see...

In [11]:
gen_data_ex = lambda : gen_data3(nobs=nobs,beta=1.5)
mc_out = monte_carlo2(num_sims,gen_data_ex,setup_shi_ex,trials=500,c1=c1)
print(mc_out)

---------
llr -271.3910547490429 -315.864093722761
omega 2.3507373547191674 319.82981756352365 381.8268362358753 1.5332114514049155 2.4686219555023166
llr -243.4534343406161 203.8819724301968
omega 2.3835471892317632 163.54175278695442 246.08949765546924 1.5438740846428387 2.161031735159634
test_stat0 250 250 -0.04355875020241465 -11.23854257081022 -11.194983820607805
test_stat 250 250 0.37211515072867724 -7.390101045961992 -7.762216196690669
---------
llr -238.19849255442847 14.273831185615563
omega 2.17234442069941 147.9002720040242 177.00930864805923 1.4738875196904986 2.0035285787903723
llr -241.50330446223472 192.78517146269587
omega 2.4921733262268613 182.62862582695323 244.9682192768428 1.5786618783725859 2.2089307106309444
test_stat0 250 250 -0.011945493535541857 -10.233210842859185 -10.221265349323643
test_stat 250 250 -0.396596443535984 -7.8707697382775335 -7.474173294741549
---------
llr -248.92772583792782 -325.16590927965217
omega 2.4980290240245986 282.88908562708724 190.

In [12]:
gen_data_ex = lambda : gen_data3(nobs=500,beta=1.5)
mc_out = monte_carlo2(num_sims,gen_data_ex,setup_shi_ex,trials=500,c1=c1)
print(mc_out)

---------
llr -599.3002716137411 -344.6066978622657
omega 5.733893150229133 287.64141991366677 268.8588570924835 2.3945548960567042 3.0149268399552915
llr -591.161625577925 -368.3345447539032
omega 5.826777718234781 289.5066715659887 323.8822287792375 2.413871934928359 3.0783761939997505
test_stat0 500 500 -0.0166217828101577 -11.209316913493677 -11.192695130683518
test_stat 500 500 0.03169954673751284 -9.3690761382579 -9.400775684995414
---------
llr -549.8625300814891 142.87735705590603
omega 3.169105078822509 245.83705687668757 317.2465161539491 1.7801980448316723 2.510265613806225
llr -551.5784200374547 -512.7530109128465
omega 3.367531199933703 308.61349330241404 244.1098905242438 1.8350834313277702 2.5432068809006405
test_stat0 500 500 -0.021294234049346314 -13.834701108771442 -13.813406874722094
test_stat 500 500 -0.8197487424762593 -10.36122228753376 -9.5414735450575
---------
llr -560.3447215384317 -868.7309681209204
omega 2.341765540726091 404.2868356466503 249.2751617145449 